In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/motion planning/

/content/drive/MyDrive/motion planning


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

import pandas as pd
import os
import random

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


# Data save

In [ ]:
path = './0817/'
np.random.seed(10)
size = 20000
kp_list = []
idx = 0
train_test_ratio = 0.8

for i in range(3, 6):
    for j in range(1, 4):
        if j % 50 == 0:
            print(i, j)
        kappa = np.array(pd.read_csv(path+'{}_{}_k.csv'.format(i, j), header=None))
        phi = np.array(pd.read_csv(path+'{}_{}_p.csv'.format(i, j), header=None))
        s = np.array(pd.read_csv(path+'{}_{}_s.csv'.format(i, j), header=None))
        temp = np.zeros((s.shape[0], s.shape[1]+1))
        rnd = np.random.choice(len(kappa), size, replace=False)

        for k in range(len(s)):
            if s[k][4] == 10:
                s[k][4] = 0
            elif s[k][4] == 50:
                s[k][4] = 1
            elif s[k][4] == 100:
                s[k][4] = 2
            elif s[k][4] == 200:
                s[k][4] = 3
            elif s[k][4] == 300:
                s[k][4] = 4
            elif s[k][4] == 400:
                s[k][4] = 5
            elif s[k][4] == 500:
                s[k][4] = 6
            
            temp[k] = np.append(s[k], np.array([idx]))

        np.random.shuffle(temp)
        train_temp, test_temp = np.split(temp, [int(train_test_ratio*len(temp))])
        
        if idx == 0:
            train_set = train_temp
            test_set = test_temp
        else:
            train_set = np.append(train_set, train_temp, axis=0)
            test_set = np.append(test_set, test_temp, axis=0)

        idx += 1
        kp = []

        for l in rnd:
            tmp = []
            k = kappa[l]
            p = phi[l]
            k = list(k)
            k.extend(list(p))
            kp.append(k)

        kp = torch.tensor(kp, dtype=torch.float).to(device)
        kp = kp.view((1, 1, size, 3))
        kp_list.append(kp)

# kp_list = np.array(kp_list)
# np.save('./kp_list', kp_list)

# np.random.shuffle(train_set)
print(train_set)
print(train_set.shape, test_set.shape)

[[-1.37204912e+00  6.35453137e-01  2.28449575e+00  1.82196274e+00
   3.00000000e+00  0.00000000e+00]
 [-2.28754061e+00 -2.82807367e+00 -2.79237705e+00  3.24517880e-01
   0.00000000e+00  0.00000000e+00]
 [-1.55097889e+00 -1.62139176e+00  2.44396250e+00 -3.90230858e-01
   3.00000000e+00  0.00000000e+00]
 ...
 [ 3.45127381e-01 -2.21081933e+00  2.33009882e+00 -3.30826654e-03
   0.00000000e+00  8.00000000e+00]
 [ 1.08757893e+00  1.40933632e+00  1.91158883e+00  1.90845374e+00
   0.00000000e+00  8.00000000e+00]
 [-5.36439146e-01 -1.34974944e+00 -2.60873297e-01  2.75435847e+00
   1.00000000e+00  8.00000000e+00]]
(21030, 6) (5261, 6)


In [ ]:
train_df = pd.DataFrame(train_set)
test_df = pd.DataFrame(test_set)

train_df.to_csv('./train.csv', index=False, header=False)
test_df.to_csv('./test.csv', index=False, header=False)

# Data load

In [ ]:
size = 20000
train_set = np.array(pd.read_csv('./20000/train.csv', header=None))
test_set = np.array(pd.read_csv('./20000/test.csv', header=None))
temp_list = np.load('./20000/kp_list.npy')

kp_list = []
for i in range(len(temp_list)):
    kp = torch.tensor(temp_list[i], dtype=torch.float).to(device)
    kp = kp.view((1, 1, size, 3))
    kp_list.append(kp)
print(kp_list)

# Model

In [ ]:
class Custom_Dataset(Dataset):
    def __init__(self, state):
        self.x = state[:, :4]
        self.target = state[:, 4]
        self.idx = state[:, 5]

    def __len__(self):
        return len(self.target)

    def __getitem__(self, index):
        x = torch.tensor(self.x[index]).float()
        y = torch.tensor(self.target[index]).float()
        idx = torch.tensor(self.idx[index]).int()
        
        return x, y, idx

In [ ]:
class NN(nn.Module):
    def __init__(self, h1, h2, h3, h4, h5, h6, output):
        super(NN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, (1, 3)),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 8, 1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 1, 1),
            nn.BatchNorm2d(1),
            nn.ReLU(),
        )

        self.hidden1 = nn.Linear(size+4, h1)
        self.hidden2 = nn.Linear(h1, h2)
        self.hidden3 = nn.Linear(h2, h3)
        self.hidden4 = nn.Linear(h3, h4)
        self.hidden5 = nn.Linear(h4, h5)
        self.hidden6 = nn.Linear(h5, h6)
        self.predict = nn.Linear(h6, output)

        # nn.init.xavier_uniform_(self.conv.weight)
        # nn.init.xavier_uniform_(self.hidden1.weight)
        # nn.init.xavier_uniform_(self.hidden2.weight)
        # nn.init.xavier_uniform_(self.hidden3.weight)
        # nn.init.xavier_uniform_(self.predict.weight)


    def forward(self, x1, x2):
        x2 = x2.view((-1,4))

        x1 = self.conv(x1)
        x1 = x1.view((-1,size))
        v, i = torch.topk(x1, 10)

        x = torch.cat([x1, x2], dim=1)

        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = F.relu(self.hidden5(x))
        x = F.relu(self.hidden6(x))
        x = F.relu(self.predict(x))

        return x


In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE ,self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, (1, 3)),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 8, 1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 1, 1),
            nn.BatchNorm2d(1),
            nn.ReLU(),
        )
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(1, 8, 1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 8, 1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 1, (1, 3)),
            nn.BatchNorm2d(1),
            nn.ReLU(),
        )
        self.fc1 = nn.Linear(size, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc31 = nn.Linear(h2, h3)
        self.fc32 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, h2)
        self.fc5 = nn.Linear(h2, h1)
        self.fc6 = nn.Linear(h1, size)


    def encode(self, x):
        x = self.conv(x)
        x = x.view((-1,size))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return self.fc31(x), self.fc32(x)


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    
    def decode(self, z):
        z = F.relu(self.fc4(z))
        z = F.relu(self.fc5(z))
        z = F.sigmoid(self.fc6(z))

        return z


    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
# 수정해야됨
def vae_loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [ ]:
h1 = 512
h2 = 512
h3 = 512
h4 = 512
h5 = 512
h6 = 512


model = NN(h1, h2, h3, h4, h5, h6, output=7).to(device)
loss_func = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0)
lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-8, max_lr=0.0001, step_size_up=1, step_size_down=19, gamma=0.95, mode='exp_range' , cycle_momentum=False)

# Training

In [ ]:
def test(model, mode, verbose=False):
    model.eval()

    correct = 0
    
    if mode == "val":
        loader = val_loader
        dataset = val_dataset
    elif mode == "test":
        loader = test_loader
        dataset = test_dataset

    with torch.no_grad():
        
        for x, y, idx in loader:
            error = []
            truth = []
            x, y = x.float().to(device), y.to(device)
            for n, index in enumerate(idx):
                if n == 0:
                    kps = kp_list[index]
                else:
                    kps = torch.cat([kps, kp_list[index]], dim=0)

            predict = model(kps, x)
            predict = torch.argmax(predict, 1)

            if verbose:
                print("Predict : ", predict)
                print("Target : ", y)
            
            correct += (predict == y).sum().item()
    
    accuracy = 100 * float(correct) / len(dataset)
    print('\nTest accuracy : {:3f}%'.format(accuracy))

    return accuracy

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=0)

batch_size = 32

stop = False

best = []

for eps in range(10000):

    for fold_index, (t, v) in enumerate(kfold.split(train_set)):

        torch.cuda.empty_cache()
        train_data = train_set[t]
        val_data = train_set[v]

        train_dataset = Custom_Dataset(train_data)
        val_dataset = Custom_Dataset(val_data)

        train_loader = DataLoader(train_dataset,batch_size=batch_size)
        val_loader = DataLoader(val_dataset,batch_size=batch_size)

        val_acc_max = 0

        for epoch in range(3):
            model.train()
            print('\n{}_{}fold_{}th epoch'.format(eps+1, fold_index+1, epoch+1))
            # print(list(model.parameters()))
            current_lr = lr_scheduler.get_last_lr()
            print("\nCurrent learning rate : ", current_lr)
                
            for i, data in enumerate(train_loader):
                x, y, idx = data
                x, y = x.float().to(device), y.to(device)
                for n, index in enumerate(idx):
                    if n == 0:
                        kps = kp_list[index]
                    else:
                        kps = torch.cat([kps, kp_list[index]], dim=0)
                # print(kps.shape)        

                predict = model(kps, x)

                loss = loss_func(predict, y.long())
                
                optimizer.zero_grad()

                loss.backward()

                optimizer.step()

                if i % 200 == 0:
                    print('#'*100)
                    # print('Target : {}'.format(y))
                    # print('Predict : {}'.format(predict))
                    print('Trian loss : {}'.format(loss))
            
            val_acc = test(model, 'val')

            lr_scheduler.step()
            
            if val_acc_max < val_acc:
                val_acc_max = val_acc
                models = [model, val_acc_max]

            if val_acc >= 85:
                stop = True

            if stop:
                torch.save(model, f'./result/c2_{h1}_{h2}_{h3}_{h4}({val_acc}).pth')
                break
        if stop:
            break
                
        best.append(models)
    if stop:
        break
        

# Save Model(manually)

In [ ]:
best = sorted(best, key=lambda x : x[1], reverse=True)
print(best)

In [ ]:
model = best[0]
val_acc = model[1]
torch.save(model[0], f'./result/{"c2"}_{h1}_{h2}_{h3}_{h4}({val_acc}).pth')

# Test

In [ ]:
model = torch.load('./result/c1_512_512_512_512_10_512_512_512(98.63333333333334)(55).pth', map_location=torch.device(device))
print(len(test_set))
test_dataset = Custom_Dataset(test_set)
test_loader = DataLoader(test_dataset, batch_size=100)
acc = test(model, 'test', verbose=True)